In [1]:
from datascience import *
import numpy as np
import pandas as pd
import math

#These lines set up the plotting functionality and formatting.
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
#Collected earring markey surveys on Google Surveys.

survey = Table.read_table('survey_responses.csv')
#survey.show(5)

In [3]:
#Cleaning Survey table 
#Created function to remove '@' in order to merge tables with Summed later

def remove_symbol(x):
    if x[0] == '@':
        x = x.replace('@', '')
    return x
            
survey_removed_symbol = survey.apply(remove_symbol, 'Username')
survey = survey.with_column('Username', survey_removed_symbol)
survey

Timestamp,Gender,Age Range,Wear Regularly,Looking For,Design Preference,Jewelry Type,Comfortable Spending Fashion,Comfortable Spending Fine,Comfortable Spending Shipping?,Purchase Habits,Shop Online,Prefer Website,Care,Username
11/8/2020 15:44:19,Female,24 - 29,"Earrings, Necklaces","Style / Design, Comfort","Minimal, Simple",Fine / Semi-fine / Demi-fine jewelry -- Costlier and hig ...,Below RM 10,RM 20 - 40,Below RM5,Planned,3,4,2,I do not use a insta!!!!
11/8/2020 17:59:29,Female,18 - 23,"Earrings, Rings (not including wedding ring), Watches","Style / Design, Quality",Elegant,Fashion jewelry -- Inexpensive jewelry made out of non-p ...,RM 10 - 20,RM 60 - 80,Below RM5,Implusively / Unplanned,3,2,3,meinlee_
11/8/2020 18:21:30,Female,24 - 29,"Earrings, Necklaces",Style / Design,Silver Huggies,Fashion jewelry -- Inexpensive jewelry made out of non-p ...,RM 20 - 30,RM 20 - 40,RM 5 - 7,Implusively / Unplanned,3,2,4,themonster.bites
11/8/2020 19:20:50,Female,18 - 23,"Earrings, Rings (not including wedding ring)","Style / Design, Craftmanship","Minimal, Simple",Fashion jewelry -- Inexpensive jewelry made out of non-p ...,RM 30 - 40,RM 20 - 40,Below RM5,Implusively / Unplanned,3,3,4,y4nni_
11/8/2020 19:23:33,Female,18 - 23,Earrings,"Style / Design, Quality",Resin Handmade,Fashion jewelry -- Inexpensive jewelry made out of non-p ...,Below RM 10,RM 20 - 40,RM 5 - 7,Implusively / Unplanned,5,3,5,shalini__29
11/8/2020 19:32:20,Female,18 - 23,"Earrings, Necklaces, Rings (not including wedding ring), ...","Style / Design, Quality",Silver Huggies,Fashion jewelry -- Inexpensive jewelry made out of non-p ...,RM 20 - 30,RM 40 - 60,RM 5 - 7,Implusively / Unplanned,2,1,2,v1ckychong
11/8/2020 19:33:45,Female,24 - 29,"Earrings, Necklaces, Rings (not including wedding ring), ...","Style / Design, Quality",Mismatch,Fashion jewelry -- Inexpensive jewelry made out of non-p ...,RM 10 - 20,RM 20 - 40,Below RM5,Planned,5,2,2,hannahreshma
11/8/2020 19:39:25,Female,18 - 23,"Earrings, Necklaces, Rings (not including wedding ring)","Style / Design, Quality, Comfort, Price","Minimal, Simple",Fine / Semi-fine / Demi-fine jewelry -- Costlier and hig ...,I do not purchase fashion earrings at all,RM 20 - 40,Below RM5,Implusively / Unplanned,5,3,5,edelnotsowise
11/8/2020 19:40:51,Female,18 - 23,"Earrings, Necklaces, Rings (not including wedding ring)","Style / Design, Quality",Elegant,Fine / Semi-fine / Demi-fine jewelry -- Costlier and hig ...,RM 20 - 30,RM 20 - 40,RM 5 - 7,Implusively / Unplanned,4,3,5,phoenixamk
11/8/2020 19:41:03,Female,18 - 23,"Earrings, Necklaces, Rings (not including wedding ring), ...","Style / Design, Quality","Statement, Bold",Handcrafted / Handmade jewelry -- Artisan jewelry assemb ...,Below RM 10,RM 40 - 60,RM 5 - 7,Planned,5,1,4,joeyxory


In [4]:
#CONFUSION: no idea how to drop NA with numpy. But not needed in this case because I plan to drop rows with 0 counts anyways 

comments = Table().read_table('GIVEAWAY_COMMENTS.csv')
comments.show(5)

﻿Users
y4nni_@yanerrim 🔥
2w2 likesReply
View replies (2)
nan
nan


In [5]:
#1 Count for every tag -- indicated by '@' character
#outer for loop: each row item, inner for loop: each character

arr_comments= comments.column(0)  #CONFUSION: using column('Users') produced error. Why?
count_tags = make_array()
tags = make_array()
users = make_array()

for comment in arr_comments:
    i,count =0,0
    for i in range(len(comment)):
        if comment[i] =='@':
            if count == 0:   #Saved string index at first '@'
                saved = i    #This is to account for users tagging multiple accounts in a single comment
            count += 1
            
    count_tags = np.append(count_tags, count)
    tags = np.append(tags, comment[saved:])    #List slicing to separate user and tagged accounts
    users = np.append(users, comment[:saved].lower()) #converted all user strings to lowercase to avoid duplicates
    
#New, cleaned table with updated counts and removed null values   
comments_cleaned = Table().with_columns('Users', users, 'Tags', tags, 'Count', count_tags).where('Count', are.above(0))
comments_cleaned.show(5)

Users,Tags,Count
y4nni_,@yanerrim 🔥,1
v1ckychong,@meinlee_ does tagging u count,1
edelnotsowise,@edellweiss___ ❤️,1
phoenixamk,@leongchewlee @vaishnavichandraraj @sabrinahsnol @alymkr ...,5
_yixuanlee,@trishalowww @xminxz @bernice_nys @kiara.lwy @joeyylaww,5


In [6]:
#This cell is for checking comments_cleaned
#Following line should have 9 rows 
comments_cleaned.where('Users', 'xtsxt').num_rows

9

In [7]:
#Table to sum the amount of tags per user
#This is to account for users tagging multiple accounts in multiple comments 

summed = comments_cleaned.drop('Tags').group('Users', sum)
summed.show(236)

Users,Count sum
00nuj17,6
_.umx01,3
_kai_sian_,1
_siew_later_,10
_strawberry_sundae,7
_yixuanlee,5
ac_5327,3
ah_jaynee,4
aikchien,3
ajlsw2331,6


In [8]:
participant_comment = summed.column('Users') #submission from comments
participants = survey.column('Username')     #submission from survey

count = summed.column('Count sum')                 #count entried from comments
ones = [1 for i in np.arange(len(participants))]   #x1 entry from survey

#participant_comment
#participants

In [9]:
#Concatenating arrays instead of joining tables to avoid losing information
con_name = np.concatenate((participant_comment, participants))
con_count = np.concatenate((count, ones))  

concatenated = Table().with_columns('Participant', con_name, 'Entry', con_count)
grouped = concatenated.group('Participant', sum)
sum_so_far_arr = np.cumsum(grouped.column('Entry sum'))
grouped = grouped.with_column('Sum So Far', sum_so_far_arr)

grouped.show(grouped.num_rows) #show all

Participant,Entry sum,Sum So Far
00nuj17,7,7
0146372663,1,8
Ah_jaynee,1,9
Anderson_ting_kai,1,10
Crystaljoya_,1,11
Erinaa29,1,12
Fen_2011,1,13
Francine_cjy,1,14
Hi bij I miss you,1,15
I do not use a insta!!!!,1,16


In [33]:
#Generate a randomized number 
winners = np.random.choice(np.arange(1,1313), 3)
for w in winners:
    for i in grouped.column('Sum So Far'):
        if i >= w:
            print(grouped.where('Sum So Far', i).column('Participant').item(0))
            break           #only print the first participant name that matches condition above 
    


#winners = np.array([ 545, 1178,  424])


j0ey.k
irene.pua1
piikaa._


# REFERENCES

DATA8 NUMPY Table Functions & Methods Reference Sheet
http://data8.org/fa20/python-reference.html
    
DATAGOOD PANDAS Cheat Sheet 
https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf 

TOWARDSDATASCIENCE.COM